Series GSE125088		Query DataSets for GSE125088
Status	Public on Jan 16, 2019
Title	Characterization of embryonic osteoclast precursors
Organism	Mus musculus
Experiment type	Expression profiling by high throughput sequencing
Summary	Embryonically established osteoclast precursors expressing tdTomato reporter gene were analyzed by single cell RNA-sequencing. Data showed that hematopoietic stem cell independent yolk-sac erythromyeloid progenitors produced embryonic osteoclast precursors.
 	
Overall design	Macrophage/ monocyte lineage cells isolated from E14.5 whole body were subjected to the sc RNA-seq. All samples included tdTomato expressing cells derived from E8.5 or E9.5 erythromyeloid progenitors in the yolk-sac.

In [2]:
import pandas as pd
import gzip
import scipy
import anndata as ad
import pybiomart
import scanpy as sc

In [3]:
barcodes = pd.read_csv("/home/roger/project_LYPI/data/GSE125088_Cx3cr1_Csf1r_new.barcodes.tsv.gz", sep="\t", compression="gzip", header=None)
features = pd.read_csv("/home/roger/project_LYPI/data/GSE125088_Cx3cr1_Csf1r_new.features.tsv.gz", sep="\t", compression="gzip", header=None)

with gzip.open("/home/roger/project_LYPI/data/GSE125088_Cx3cr1_Csf1r_new.matrix.mtx.gz", "rt") as f:
    sparse_matrix = scipy.io.mmread(f)
matrix = sparse_matrix.todense()

In [4]:
matrix.shape

(31054, 39048)

In [5]:
# Create the AnnData object. # They do not have metadata????
adata = ad.AnnData(X=matrix.T)
adata.obs_names = barcodes.iloc[:, 0].values
adata.var_names = features.iloc[:, 1].values

In [ ]:
# Basic pre-processing steps   
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=1)

In [ ]:
sc.pl.umap(adata, color=["Vdr", "Cyp27a1",  "Cyp2r1",  "leiden"], legend_loc="on data", frameon=False, ncols=5)

In [ ]:
sc.pl.violin(adata, keys=["Vdr", "Cyp27a1", "Cyp27b1", "Cyp2r1"], groupby="leiden") 

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="leiden", method="wilcoxon")

In [ ]:
#sampling
# Genes to check
genes_to_check = ["Vdr", "Cyp27a1", "Cyp27b1", "Cyp2r1"]

# Extract the DE results
rank_genes_groups = adata.uns["rank_genes_groups"]

# Initialize a dictionary to store the results
significant_genes = {}

# Check each gene in the results
for gene in genes_to_check:
    significant_genes[gene] = {}
    for group in rank_genes_groups['pvals_adj'].dtype.names:
        # Check if the gene is in the names for the current group
        if gene in rank_genes_groups['names'][group]:
            # Get the index of the gene in the names array
            gene_index = list(rank_genes_groups['names'][group]).index(gene)
            # Get the corresponding p-value
            pval = rank_genes_groups['pvals_adj'][group][gene_index]
            # Check if the p-value is below the threshold
            if pval < 0.05:
                significant_genes[gene][group] = pval

# Display the results
for gene, groups in significant_genes.items():
    if groups:
        print(f"{gene} is significant in the following groups (p < 0.05):")
        for group, pval in groups.items():
            print(f"  - {group}: p-value = {pval}")
    else:
        print(f"{gene} is not significant in any group (p < 0.05).")
# 

In [ ]:
import decoupler as dc
net = dc.get_collectri(organism='human', split_complexes=False)
new_adata = adata
new_adata.var_names = new_adata.var_names.str.upper()
dc.run_ulm(
    mat=new_adata,
    net=net,
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False
)
new_adata.obsm['collectri_ulm_estimate'] = new_adata.obsm['ulm_estimate'].copy()
new_adata.obsm['collectri_ulm_pvals'] = new_adata.obsm['ulm_pvals'].copy()
acts = dc.get_acts(new_adata, obsm_key='ulm_estimate')

In [ ]:
sc.pl.umap(acts, color=['VDR', 'leiden'], cmap='RdBu_r', vcenter=0)
sc.pl.violin(acts, keys=['VDR'], groupby='leiden', rotation=90)